<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/instagramm_cookes_mail_25022019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Получение фото

In [0]:
import hashlib
import json
import re
import requests


count=36
session = requests.Session()

      
def update(obj=None, pointer=None, idx=None, rhx_gis=None, settings={}):

      src_adress = [] 
      base_url = ""
      query = "https://www.instagram.com/"
      if not obj is None:
            query += base_url + obj
            
        
      if pointer is None: 
        response = session.get(query, **settings)
        match = re.search(r"<script[^>]*>\s*window._sharedData\s*=\s*((?!<script>).*)\s*;\s*</script>", response.text)
        data = json.loads(match.group(1))
        rhx_gis = data["rhx_gis"]
        csrf_token = data["config"]["csrf_token"]
        idx = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["id"]
        
        #print("end_cursor", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])
        #print("'has_next_page", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'])
        
        for number in range(len(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"])):
	          src_adress.append(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"][number]["node"]["thumbnail_resources"][4]["src"])
        if data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']:                 
            pointer = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        else:
            pointer = None

        return rhx_gis, pointer, src_adress, idx, csrf_token

      
      else:
        variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        while True:
            data = {"id": idx, "first": 36, "after": pointer}
            variables=variables_string.format(**data)
            query_hash="c6809c9c025875ac6f02619eae97a80e"
            #response = graphql_request(query_hash="c6809c9c025875ac6f02619eae97a80e", variables=variables, settings=settings)
            settings["params"] = {"query_hash": query_hash}
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            response = session.get("https://www.instagram.com/graphql/query/", **settings)
            response.raise_for_status()
            
            data = response.json()["data"]["user"]["edge_owner_to_timeline_media"]

            edges = data["edges"]
            page_info = data["page_info"]
            length = len(edges)

            for index in range(length):
                    node = edges[index]["node"]
                    shortcode = node["shortcode"]        #["src"]
                    if not "shortcode" in node:
                        length -= 1
                        continue
                    m = node["display_url"]
                    #m._set_data(node)
                    src_adress.append(m)
                    #print("node", node)
                
            if page_info["has_next_page"]:
                    
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None

            return src_adress, pointer

      

In [0]:
src_adress = []
rhx_gis, pointer, src, idx, csrf_token = update("likekate_")
src_adress.extend(src)
print("rhx_gis:", rhx_gis, "csrf_token:", csrf_token, "idx =", idx)
for i in range(100):
  src, pointer = update("likekate_", pointer=pointer, idx=idx, rhx_gis=rhx_gis)
  src_adress.extend(src)
  print(len(src_adress))


# Авторизация

In [0]:
import hashlib
import json
import pickle
import re
import requests
session = requests.Session()
# rhx_gis: fc2e73d4fd7dddcd31d28bea5cb2df59 csrf_token: 23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv
def AgentAccount(login=None, password=None, settings={}, load=False):
    # load cookie    
        if load:
           print("1")
           LOGIN_URL = "https://www.instagram.com/accounts/login/ajax/"
           with open('cookie.txt', 'rb') as s:
              #session = session.cookies.update(pickle.load(s))
              cookies = requests.utils.cookiejar_from_dict(pickle.load(s))
              #pickle.dump(a, s)
              #cookies = requests.utils.cookiejar_from_dict(pickle.load(s))
              session.cookies.update(cookies)
              #session = session(cookies=cookies) 
              #cookies = pickle.load(s)
              #if cookies:
              #jar = requests.cookies.RequestsCookieJar()
              #jar._cookies = cookies
              #session.cookies = jar
              print("session.cookies", session.cookies)
              #session.hooks['response'].append(cookie)
              #csrf_token = "2BYlKYXjE31YtbbT2KB8mRJ16ikszq14"
              #csrf_token = data[csrftoken]
              csrf_token = session.cookies['csrftoken']
              #data = {"username": login, "password": password}
        
              headers = {
                    "X-CSRFToken": csrf_token,
                    "referer": "https://www.instagram.com/",
                       }
              response = session.post(LOGIN_URL, headers=headers, cookies=cookies)
              #response = session.post("https://www.instagram.com/accounts/login/ajax/")
              response.raise_for_status()
              session.cookies.update(response.cookies)
              
        else:
            print("2")
            csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
            login = "melvynasarterca"
            password = "00pp0099"
            data = {"username": login, "password": password}
        
            if "headers" in settings:
                #settings["params"] = {"query_hash": query_hash}
                settings["headers"]["X-CSRFToken"] = csrf_token
                settings["headers"]["referer"] = "https://www.instagram.com/"
            else:
                settings["headers"] = {
                    "X-CSRFToken": csrf_token,
                    "referer": "https://www.instagram.com/",
                       }

            response = session.post("https://www.instagram.com/accounts/login/ajax/", data, **settings)
            #response.raise_for_status()
            
            
            resp = response.json()
            field = 'checkpoint_url'
            
            if field in resp:
              print("11111")
              resp = response.json()[field]
              resp = "https://www.instagram.com" + resp
              data = {"choice": 1}
              response = session.post(resp, data, **settings)
              #print(response.text)
              response.raise_for_status()
              code = input('Enter code received: ')
              code_data = {'security_code': code}
              
              response = session.post(resp, code_data, **settings, allow_redirects=True)
              session.headers.update({'X-CSRFToken': response.cookies['csrftoken']})
              print("response")
            
            print("finish")

            # Save cookie
            with open("cookie.txt", 'wb') as fout:
              #f.truncate()
              pickle.dump(requests.utils.dict_from_cookiejar(session.cookies), fout)  #session.cookies._cookies
              print("f")
            print("response2:", response)
            try:
                data = response.json()
                if not data["authenticated"]:
                  raise AuthException(login) 
            except (ValueError, KeyError) as exception:
                 print("ERROR1")

AgentAccount(load=True)
#AgentAccount()

1
session.cookies <RequestsCookieJar[<Cookie csrftoken=se2F5mbejVPYiI8MtX0ZVHewm9XD3XcG for />, <Cookie ds_user_id=10387214183 for />, <Cookie mid=XG5wpgAEAAGYzCWUwh6LbXSpJBKT for />, <Cookie rur=FRC for />, <Cookie sessionid=10387214183%3AtenEsHzENLxy6s%3A2 for />, <Cookie urlgen="{35.233.191.48: 15169}:1gwkkR:DgDKxYW34xLYxx_DFs9RSMyuWI4" for />]>


# Получение кода из почты

In [0]:
#now = datetime.now()
import imaplib
#from datetime import datetime

import datetime
now = datetime.datetime.now()
mail = imaplib.IMAP4_SSL('imap.mail.ru')
mail.login('rogeca@bk.ru', '66yy6677')
mail.select("inbox")
import email
import re

result, data = mail.uid('search', None, "ALL") # Выполняет поиск и возвращает UID писем.
latest_email_uid = data[0].split()[-1]
result, data = mail.uid('fetch', latest_email_uid, '(RFC822)')
email_body = data[0][1]
mail = email.message_from_bytes(email_body)
mail_from = mail['From']
#pattern = re.compile('([security@mail.instagram.com]).')
'''
mail_from = re.findall('(security@mail.instagram.com).', mail_from)[0]
mail_date = mail['Date']
mail_date = str(mail_date)
mail_date = re.findall('\w{3}.\s(\d{2}\s\w{3}\s\d{4}\s\d{2}:\d{2}:\d{2})', mail_date)[0]
mail_date = datetime.datetime.strptime(mail_date, '%d %b %Y %H:%M:%S')
'''
data = str(mail)
security_code = re.findall('[0-9]{6}', data)[9]


mail_from = email.utils.parseaddr(mail['From'])[1]
mail_date = email.utils.parsedate_to_datetime(mail['Date'])
mail_date = mail_date.replace(tzinfo=None)

period = mail_date - now
print("{} дней  {} секунд".format(period.days, period.seconds))
if period.seconds > 30:
  print("true")
if mail_date < now:
  print("mail_date < now")
elif mail_date > now:
  print("mail_date > now")
if mail_from == 'security@mail.instagram.com':
  print("Адрес отправителя правильный")
print("security_code", security_code)
print("maildate", mail_date)  
print("mail_from", mail_from)  
#result2 print("mail_from", mail_from)= re.findall('[Date:]\s\w{3}.\s(\d{1-2}\s\w{3}\s\d{4}\s\d{2}:\d{2}:\d{2})', date)

-2 дней  40330 секунд
true
mail_date < now
Адрес отправителя правильный
security_code 214976
maildate 2019-02-21 00:36:50
mail_from security@mail.instagram.com


In [0]:
from datetime import datetime
now = datetime.now()
datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M")
print(datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M"))

2019.02.22 10:34


In [0]:
from datetime import datetime
now = datetime.now()
period = d - now
print("Всего: {} секунд".format(period.total_seconds()))
print("{} дней  {} секунд".format(period.days, period.seconds))


Всего: -122734.352263 секунд
-2 дней  50065 секунд


# Сохранение cookies на компьютер

In [0]:
import os
#session.close()
#response.raise_for_status()
#session
print( os.listdir('/content') )
from google.colab import files
files.download('cookie.txt') 

['.config', 'cookie.txt', 'sample_data']


# Загрузка Cookie на сервер

In [0]:
from google.colab import files
files.upload()

Saving cookie.txt to cookie.txt


{'cookie.txt': b'\x80\x03}q\x00(X\t\x00\x00\x00csrftokenq\x01X \x00\x00\x00se2F5mbejVPYiI8MtX0ZVHewm9XD3XcGq\x02X\n\x00\x00\x00ds_user_idq\x03X\x0b\x00\x00\x0010387214183q\x04X\x03\x00\x00\x00midq\x05X\x1c\x00\x00\x00XG5wpgAEAAGYzCWUwh6LbXSpJBKTq\x06X\x03\x00\x00\x00rurq\x07X\x03\x00\x00\x00FRCq\x08X\t\x00\x00\x00sessionidq\tX \x00\x00\x0010387214183%3AtenEsHzENLxy6s%3A2q\nX\x06\x00\x00\x00urlgenq\x0bX;\x00\x00\x00"{35.233.191.48: 15169}:1gwkkR:DgDKxYW34xLYxx_DFs9RSMyuWI4"q\x0cu.'}

# Проверка директории

In [0]:
import os
print( os.listdir('/content') )

['.config', 'sample_data']


# Получение подписчиков


In [3]:
#query_hash:56066f031e6239f35a904ac20c9f37d9
#variables:{"id":"10180157718","include_reel":true,"fetch_mutual":false,"first":12,"after":""}

import hashlib
from time import sleep
from requests.exceptions import HTTPError
def get_followers(account=None, pointer=None, count=100, settings={}, limit=50):
  
        rhx_gis, pointer, src_adress, idx, csrf_token = update(account, settings)
        #csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
        #rhx_gis = "fc2e73d4fd7dddcd31d28bea5cb2df59"
        #query_hash = "QVFENklrNmY2Y2JvRGhEa2k5UV9XZTVkRHcwSDB6OXM3dWhhZkVsSXpNdlZacEJQVU9QVkcybDQtaC11c04tSXN3cWlmUzVIN3NxeG1BR2phV0VYSU1fcw=="
        query_hash="37479f2b8209594dde7facb0d904896a"
        #idx = "2120762022"

        if pointer is None:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first}}}'
            variables_string = '{{"id":"{id}","first":{first}}}'
        else:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first},"after":"{after}"}}'
            variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        followers = []

        while True:
            data = {"first": min(limit, count), "id": idx}
            if not pointer is None:
                data["after"] = pointer
            settings["params"] = {"query_hash": query_hash}
            variables=variables_string.format(**data)
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            #response = session.get("https://www.instagram.com/graphql/query/", **settings)
            #response.raise_for_status()
            
            
           
            for i in range(0, 3):
              try:
                response = session.get("https://www.instagram.com/graphql/query/", **settings)
                print(response.headers)
                response.raise_for_status()

              except (HTTPError, ValueError, KeyError) as exception:

                    print("ERROR", exception.response.status_code)
  
                    print("sleep") 
                    sleep(6) 
                    print("end sleep")
                    continue
              break
            #data = response.json()
            data = response.json()["data"]["user"]["edge_followed_by"]
            edges = data["edges"]
            page_info = data["page_info"]
            #print(page_info)
            followers_count = data["count"]
                
            for index in range(min(len(edges), count)):
                    node = edges[index]["node"]
                    a = node["username"]
                    id = node["id"]
                    profile_pic_url = node["profile_pic_url"]
                    is_verified = node["is_verified"]
                    full_name = node["full_name"]
                    followers.append(a)
                    if len(followers) % 100 == 0:
                      print(len(followers))
                
            if page_info["has_next_page"]:
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None
                
            if len(edges) < count and page_info["has_next_page"]:
                    count = count-len(edges)
                    variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
            else:
                    return followers, pointer
            
                
get_followers("likekate_")

HTTPError: ignored

In [0]:
import hashlib
import json
import re
import requests


count=36
session = requests.Session()

      
def update(obj=None, pointer=None, idx=None, rhx_gis=None, settings={}):

      src_short = [] 
      #shortcodes = []
      base_url = ""
      query = "https://www.instagram.com/"
      if not obj is None:
            query += base_url + obj
            
        
      if pointer is None: 
        response = session.get(query, **settings)
        match = re.search(r"<script[^>]*>\s*window._sharedData\s*=\s*((?!<script>).*)\s*;\s*</script>", response.text)
        data = json.loads(match.group(1))
        rhx_gis = data["rhx_gis"]
        csrf_token = data["config"]["csrf_token"]
        idx = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["id"]
        
        #print("end_cursor", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])
        #print("'has_next_page", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'])
        
        for number in range(len(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"])):
            src_adress = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"][number]["node"]["thumbnail_resources"][4]["src"]
            shortcodes = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"][number]["node"]["shortcode"]
            src_short.append([src_adress, shortcodes]) 
        if data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']:                 
            pointer = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        else:
            pointer = None

        return rhx_gis, pointer, src_short, idx, csrf_token

      
      else:
        variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        while True:
            data = {"id": idx, "first": 36, "after": pointer}
            variables=variables_string.format(**data)
            query_hash="c6809c9c025875ac6f02619eae97a80e"
            #response = graphql_request(query_hash="c6809c9c025875ac6f02619eae97a80e", variables=variables, settings=settings)
            settings["params"] = {"query_hash": query_hash}
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            response = session.get("https://www.instagram.com/graphql/query/", **settings)
            response.raise_for_status()
            
            data = response.json()["data"]["user"]["edge_owner_to_timeline_media"]

            edges = data["edges"]
            page_info = data["page_info"]
            length = len(edges)

            for index in range(length):
                    node = edges[index]["node"]
                    shortcode = node["shortcode"]
                    shortcodes.append(shortcode)
                    if not "shortcode" in node:
                        length -= 1
                        continue
                    m = node["display_url"]
                    #m._set_data(node)
                    src_adress.append(m)
                    #print("node", node)
                
            if page_info["has_next_page"]:
                    
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None

            return src_adress, pointer, shortcodes

      

In [48]:
src_short = []
rhx_gis, pointer, src_short, idx, csrf_token = update("likekate_")
print("src_short=", src_short, "rhx_gis:", rhx_gis, "csrf_token:", csrf_token, "idx =", idx)


src_short= [['https://scontent-sea1-1.cdninstagram.com/vp/3516c6aeacc162a65b9cddc890356bee/5D134718/t51.2885-15/sh0.08/e35/c0.135.1080.1080/s640x640/52995640_355780601730094_7074792300884570742_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvTUR1YFccW'], ['https://scontent-sea1-1.cdninstagram.com/vp/a7ba11585a470c54702a23c5709a4301/5D11016E/t51.2885-15/sh0.08/e35/c0.135.1080.1080/s640x640/54771063_1177727545734749_7065054912353896229_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvSPLldFrdp'], ['https://scontent-sea1-1.cdninstagram.com/vp/a73cb072f9a0048d77af38aa7fa7bed7/5D4A89BA/t51.2885-15/sh0.08/e35/c0.76.1080.1080/s640x640/53554839_261463721408878_170681902103516927_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvRtsWZF-aU'], ['https://scontent-sea1-1.cdninstagram.com/vp/72d26b78230b3c0d46678ca2af13a3f4/5D103DFD/t51.2885-15/sh0.08/e35/c99.0.882.882/s640x640/54277232_427032621398898_5109271456331085324_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvOwk_Rl7Cu'], ['https:

In [8]:
from requests.exceptions import HTTPError, ConnectionError
try:
                response = session.get("https://www.instagram.com/graphql/query/", proxies={"http": proxy, "https": proxy}, timeout=5.0, **settings)
                #print(response.headers)
                response.raise_for_status()

              except (HTTPError, ValueError, KeyError, ConnectionError) as exception:

                    #print("ERROR", exception.response.status_code)
  
                    print("sleep") 
                    #sleep(6) 
                    proxy = next(proxy_pool)
                    print("proxy", proxy)
                    print("end sleep")
                    continue
              break







import requests
url = 'https://httpbin.org/ip'
proxies = {
    "http": 'http://36.91.129.164:80', 
    "https": 'http://36.91.129.164:80'
}
try:
    response = requests.get(url,proxies=proxies)
    print(response.json())
except (HTTPError, ValueError, KeyError, ConnectionError) as exception:

       #print("ERROR", exception.response.status_code)
           

{'origin': '36.91.129.164, 36.91.129.164'}


In [0]:
import requests
from itertools import cycle
import traceback
proxies = {
    '177.72.164.164:53281',
    '36.91.129.164:80',
}

proxy_pool = cycle(proxies)



#url = 'https://httpbin.org/ip'
#response = requests.get(url,proxies=proxies)
#print(response.json())

#151.11.204.141:45891

#177.72.164.164:53281

#120.92.174.37:1080

#36.91.129.164:80

#36.66.210.211:31437



In [19]:
from requests.exceptions import HTTPError, ConnectionError

def get_likes(account=None, pointer=None, count=6000, settings={}, limit=24):
        src_short = []
        rhx_gis, _, src_short, idx, csrf_token = update(account)
        query_hash="e0f59e4a1c8d78d0161873bc2ee7ec44"
        shortcode = src_short[1][1]
        
        likers = []
        if pointer is None:
            
            variables_string = '{{"shortcode":"{shortcode}","include_reel":true,"first":24}}'
        else:
           
            variables_string = '{{"shortcode":"{shortcode}","include_reel":true,"first":12,"after":"{after}"}}'
        
        proxy = next(proxy_pool)
        print("proxy", proxy)
        while True:
            data = {"first": min(limit, count), "shortcode": shortcode}
            if not pointer is None:
                data["after"] = pointer
            settings["params"] = {"query_hash": query_hash}
            variables=variables_string.format(**data)
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            #response = session.get("https://www.instagram.com/graphql/query/", **settings)
            #response.raise_for_status()
            
            
            
            
            for i in range(0, 3):
              try:
                response = session.get("https://www.instagram.com/graphql/query/", proxies={"http": proxy, "https": proxy}, timeout=5.0, **settings)
                #print(response.headers)
                response.raise_for_status()

              except (HTTPError, ValueError, KeyError, ConnectionError) as exception:

                    #print("ERROR", exception.response.status_code)
  
                    print("sleep") 
                    #sleep(6) 
                    proxy = next(proxy_pool)
                    print("proxy", proxy)
                    print("end sleep")
                    continue
              break
            data = response.json()["data"]["shortcode_media"]["edge_liked_by"]
            
            #pointer = data["page_info"]["end_cursor"]
            #for number in range(len(data["edges"])):
                # likers.append(data["edges"][number]["node"]["username"])

                 
            if data["page_info"]["has_next_page"]:                 
               pointer = data["page_info"]["end_cursor"]

            else:
               pointer = None
 
            edges = data["edges"]    
            for index in range(min(len(edges), count)):
                    node = edges[index]["node"]
                    
                    likers.append(data["edges"][index]["node"]["username"])
                    
                    if len(likers) % 100 == 0:
                      print(len(likers))
                
            if data["page_info"]["has_next_page"]:
                    pointer = data["page_info"]["end_cursor"]
            else:
                    pointer = None
                
            if len(edges) < count and data["page_info"]["has_next_page"]:
                    
                    count = count-len(edges)
                    
                    variables_string = '{{"shortcode":"{shortcode}","include_reel":true,"first":12,"after":"{after}"}}'
            else:
                    return likers, pointer
            
likerss = []              
a, b = get_likes("likekate_")
print(a,b)


proxy 36.91.129.164:80
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
sleep
proxy 177.72.164.164:53281
end sleep
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
sleep
proxy 36.91.129.164:80
end sleep
sleep
proxy 177.72.164.164:53281
end sleep
sleep
proxy 36.91.129.164:80
end sleep


KeyError: ignored

In [0]:
https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/

In [42]:
a = []
a.append('2')
a.append('3')
a.append(['6', '7'])

a

['2', '3', ['6', '7']]

In [52]:
src_short= [['https://scontent-sea1-1.cdninstagram.com/vp/3516c6aeacc162a65b9cddc890356bee/5D134718/t51.2885-15/sh0.08/e35/c0.135.1080.1080/s640x640/52995640_355780601730094_7074792300884570742_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvTUR1YFccW'], ['https://scontent-sea1-1.cdninstagram.com/vp/a7ba11585a470c54702a23c5709a4301/5D11016E/t51.2885-15/sh0.08/e35/c0.135.1080.1080/s640x640/54771063_1177727545734749_7065054912353896229_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvSPLldFrdp'], ['https://scontent-sea1-1.cdninstagram.com/vp/a73cb072f9a0048d77af38aa7fa7bed7/5D4A89BA/t51.2885-15/sh0.08/e35/c0.76.1080.1080/s640x640/53554839_261463721408878_170681902103516927_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvRtsWZF-aU'], ['https://scontent-sea1-1.cdninstagram.com/vp/72d26b78230b3c0d46678ca2af13a3f4/5D103DFD/t51.2885-15/sh0.08/e35/c99.0.882.882/s640x640/54277232_427032621398898_5109271456331085324_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvOwk_Rl7Cu'], ['https://scontent-sea1-1.cdninstagram.com/vp/f11b2b2f14603c80979b4f90c3f41c39/5D2A3658/t51.2885-15/sh0.08/e35/c135.0.810.810/s640x640/54732430_388176095071801_6583190214227665416_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvOg4nNFYOG'], ['https://scontent-sea1-1.cdninstagram.com/vp/a26aa1dc4a54c58c0605234ee9405bef/5D0EF605/t51.2885-15/sh0.08/e35/c0.134.1080.1080/s640x640/54247325_416490032229931_9130389259129451996_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvLsC9yl42d'], ['https://scontent-sea1-1.cdninstagram.com/vp/9c48f5ff57e8999a7ab9f43306f2dae5/5D1B80AF/t51.2885-15/sh0.08/e35/c0.135.1080.1080a/s640x640/53759627_2722945587778377_6202263191087393961_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvKMPCZFjEu'], ['https://scontent-sea1-1.cdninstagram.com/vp/e69272f6ceec0f4546eaad290985eaa3/5D1EB2CA/t51.2885-15/sh0.08/e35/c0.89.1080.1080a/s640x640/53160614_833994840270453_6374108528409189482_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvJqIPCF32N'], ['https://scontent-sea1-1.cdninstagram.com/vp/6ff9a262d391622a2e3e480377a9cc61/5D30B497/t51.2885-15/sh0.08/e35/c0.60.1080.1080/s640x640/53599195_2298457243756244_860970036235489103_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvJLXPKl_65'], ['https://scontent-sea1-1.cdninstagram.com/vp/0707eaf83f7d690244742a8aa31d9513/5D13F635/t51.2885-15/sh0.08/e35/c110.0.859.859/s640x640/53845821_123394452095378_421088119716395280_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvHIN6-FlyL'], ['https://scontent-sea1-1.cdninstagram.com/vp/6e3dea76552f3710faae11f46319f783/5D47B163/t51.2885-15/sh0.08/e35/c0.135.1080.1080/s640x640/53495564_2105689592799562_3035824699125123754_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvGg9LyFO2P'], ['https://scontent-sea1-1.cdninstagram.com/vp/5548e3689198076f301a9758a9259832/5D1D37B1/t51.2885-15/sh0.08/e35/c0.39.1080.1080/s640x640/52849505_181784786122303_161529304944985349_n.jpg?_nc_ht=scontent-sea1-1.cdninstagram.com', 'BvFWOCKFbU9']]

src_short[1][1]

'BvSPLldFrdp'